In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",0,0.668975,"""a""",0.088677,0.03872,0.692182,0.162394,0.111069,-1.01329
0.841471,1,"""a""",0,0.98608,"""a""",0.805164,0.779672,0.443406,0.995602,0.751761,-0.310421
0.909297,2,"""a""",0,0.086925,"""a""",0.841344,0.925695,0.50395,0.338938,0.105886,-0.351961
0.14112,3,"""a""",1,0.740415,"""a""",0.052323,0.657117,0.481978,0.894739,0.061509,-0.517888
-0.756802,4,"""a""",0,0.88655,"""a""",0.285143,0.060221,0.400455,0.838866,0.223557,-0.539818


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.162394,-0.111069
-0.841471,-0.995602,-0.751761
-0.909297,-0.338938,-0.105886
-0.14112,-0.894739,-0.061509
0.756802,-0.676472,-0.112488


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

lstsq_coeffs
list[f64]
"[-0.498603, -0.33243]"


In [8]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975
str,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.21832,0.001684,129.643652,0.0,0.215019,0.221621
"""exp(x2)""",0.174057,0.000678,256.773055,0.0,0.172728,0.175386
"""sin(x3)""",-1.744024,0.001343,-1298.651671,0.0,-1.746656,-1.741391
"""__bias__""",-0.106671,0.001509,-70.708519,0.0,-0.109628,-0.103714


In [9]:
df.lazy().select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = "y", # We can either put pl.col("y") here or just the string "y"
        add_bias=False
    )
).collect()

lstsq_coeffs
list[f64]
"[-0.498603, -0.33243]"


In [10]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,lstsq_coeffs
str,list[f64]
"""a""","[-0.502418, -0.331316]"
"""a""","[-0.502418, -0.331316]"
"""a""","[-0.502418, -0.331316]"
"""a""","[-0.502418, -0.331316]"
"""a""","[-0.502418, -0.331316]"
…,…
"""b""","[-0.494669, -0.333612]"
"""b""","[-0.494669, -0.333612]"
"""b""","[-0.494669, -0.333612]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.088677,0.03872,-1.01329,-0.057086,-0.956204
0.805164,0.779672,-0.310421,-0.660643,0.350222
0.841344,0.925695,-0.351961,-0.727225,0.375264
0.052323,0.657117,-0.517888,-0.244533,-0.273355
0.285143,0.060221,-0.539818,-0.162192,-0.377626


In [12]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,lstsq_coeffs
str,list[f64]
"""a""","[-0.502418, -0.331316]"
"""b""","[-0.494669, -0.333612]"


In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,lstsq_coeffs
str,list[f64]
"""a""","[-0.341934, -0.148501]"
"""b""","[-0.322061, -0.162986]"


In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""b""",-0.530644
"""a""",-0.568997


In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-1.01329,0.088677,0.03872,null,null
-0.310421,0.805164,0.779672,null,null
-0.351961,0.841344,0.925695,null,null
-0.517888,0.052323,0.657117,null,null
-0.539818,0.285143,0.060221,"[-0.196578, -0.364183]",-0.077984
…,…,…,…,…
-0.843147,0.918913,0.012053,"[-0.996681, 0.730759]",-0.907055
0.162252,0.710394,0.434792,"[-0.981471, 1.022926]",-0.252471
-0.913019,0.013746,0.673154,"[-0.400092, -0.615027]",-0.419507


In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [17]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

a
list[f64]
"[29.033317, 28.718227, 28.553289]"


In [18]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.865458,"[-0.635054, 0.772468]"
28.567486,"[0.772468, 0.635054]"


In [19]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
-0.086046
-0.120264
-0.202165
-0.589407
-0.428749


# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.335395,1.016465,0.499,0.499599,0.499299,0.498694,0.499649
"""a""",0.337517,1.009023,0.495568,0.494174,0.49487,0.489099,0.493713


# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""to"""
"""hello"""
"""world"""
"""going"""
"""church"""


In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""world"""
""""""
"""church"""
"""go"""
"""hello"""


In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

word
str
"""banana"""


In [30]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [31]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [32]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
0.35817
0.339043
-0.702272


In [33]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,0.269329,null
null,-1.042705,null
0.35817,-0.274638,-0.516221
0.339043,0.273165,0.417973
-0.702272,-0.342035,-0.800959


In [34]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""Fp3O""",null
null,"""aHLr3""",null
0.35817,"""L""","""dc7yg"""
0.339043,"""kwDS2""","""n1"""
-0.702272,"""O""","""bcuMk"""


In [35]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
0.35817,"""6hPCW"""
0.339043,"""6arks"""
-0.702272,"""k7acd"""


In [36]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-1.250091,null,-1.250122
null,0.311516,null,0.311129
0.35817,-1.170967,0.162608,-1.170986
0.339043,0.336469,2.754069,0.336211
-0.702272,-1.20616,2.603072,-1.206293


In [37]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.143631,-0.379969,7
null,0.052108,-0.116436,9
0.35817,0.698767,-0.138163,7
0.339043,0.2052,-0.428697,9
-0.702272,0.416529,1.914601,9


In [38]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.query_ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-1.050198,0.293801,0.362033,0.834422


In [39]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.130563,0.070355,3,5
1,0.745316,0.200161,4,1
2,0.751204,0.676162,3,2
0,0.072519,0.328465,0,4
1,0.937098,0.377092,3,8


In [40]:
# In dataframe statistical tests!
df.select(
    pds.query_ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.634996,0.525446}","{40.054525,0.294962}","{2.042828,0.085674}"


In [41]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.query_ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.query_chi2("category_1", "category_2").alias("chi2-test"),
        pds.query_f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬─────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test           ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                 ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]           ┆ struct[2]           │
╞═══════════╪══════════════════════╪═════════════════════╪═════════════════════╡
│ 0         ┆ {1.329747,0.183693}  ┆ {39.81085,0.304262} ┆ {1.012865,0.39942}  │
│ 1         ┆ {-1.280025,0.200625} ┆ {38.755402,0.34643} ┆ {0.402648,0.806853} │
│ 2         ┆ {1.045479,0.295878}  ┆ {34.419632,0.54383} ┆ {0.94893,0.434649}  │
└───────────┴──────────────────────┴─────────────────────┴─────────────────────┘


In [42]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
521,0.1042
527,0.1054
577,0.1154
581,0.1162
554,0.1108
592,0.1184
538,0.1076
530,0.106
580,0.116


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [43]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [44]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.261034,0.746559,0.020062,0.730306,1.80138,9
1,0.793592,0.420272,0.97611,0.646368,7.386107,5
2,0.849105,0.856012,0.862023,0.10227,3.453491,21
3,0.324914,0.859053,0.790551,0.451571,3.880607,8
4,0.838066,0.970615,0.681921,0.551016,6.422622,14


In [45]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.261034,0.746559,0.020062,0.730306,1.80138,427
1,0.793592,0.420272,0.97611,0.646368,7.386107,291
2,0.849105,0.856012,0.862023,0.10227,3.453491,6
3,0.324914,0.859053,0.790551,0.451571,3.880607,163
4,0.838066,0.970615,0.681921,0.551016,6.422622,212


In [46]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.261034,0.746559,0.020062,0.730306,1.80138,"[0, 804, … 1468]"
1,0.793592,0.420272,0.97611,0.646368,7.386107,"[1, 1478, … 34]"
2,0.849105,0.856012,0.862023,0.10227,3.453491,"[2, 596, … 373]"
3,0.324914,0.859053,0.790551,0.451571,3.880607,"[3, 444, … 1335]"
4,0.838066,0.970615,0.681921,0.551016,6.422622,"[4, 692, … 1501]"


In [47]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬──────────────────┬────────────────────┐
│ id  ┆ best friends     ┆ best friends count │
│ --- ┆ ---              ┆ ---                │
│ u32 ┆ list[u32]        ┆ u32                │
╞═════╪══════════════════╪════════════════════╡
│ 0   ┆ [0, 804, … 1533] ┆ 6                  │
│ 1   ┆ [1, 1478, 1370]  ┆ 2                  │
│ 2   ┆ [2, 596, … 496]  ┆ 12                 │
│ 3   ┆ [3, 444, … 1335] ┆ 3                  │
│ 4   ┆ [4, 692, … 1669] ┆ 10                 │
└─────┴──────────────────┴────────────────────┘


In [48]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.261034,0.746559,0.020062,0.730306,1.80138,"[0, 804, … 1468]","[0.0, 0.054735, … 0.081026]"
1,0.793592,0.420272,0.97611,0.646368,7.386107,"[1, 1478, … 34]","[0.0, 0.083431, … 0.108403]"
2,0.849105,0.856012,0.862023,0.10227,3.453491,"[2, 596, … 373]","[0.0, 0.038438, … 0.062527]"
3,0.324914,0.859053,0.790551,0.451571,3.880607,"[3, 444, … 1335]","[0.0, 0.038474, … 0.075953]"
4,0.838066,0.970615,0.681921,0.551016,6.422622,"[4, 692, … 1501]","[0.0, 0.035573, … 0.04935]"


In [49]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
6,0.714945,0.604465,0.791423,0.120827,5.331417
7,0.628659,0.279012,0.318958,0.28031,7.426856
9,0.744796,0.462441,0.54594,0.336285,4.578712
16,0.508039,0.421699,0.706729,0.900134,6.923955
17,0.232995,0.216001,0.690997,0.674436,1.646646


In [50]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
16,0.508039,0.421699,0.706729,0.900134,6.923955
65,0.561066,0.547812,0.370431,0.951031,2.641918
90,0.55244,0.487972,0.928881,0.339141,2.622192
217,0.584801,0.495298,0.206617,0.304759,3.676434
268,0.492433,0.423682,0.726202,0.842923,0.370434


In [51]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
333,0.508817,0.468379,0.542423,0.431881,8.758285
467,0.483835,0.461752,0.107604,0.516297,8.786492
541,0.552019,0.536066,0.034837,0.967278,9.566839
580,0.530541,0.551806,0.407562,0.450443,8.866895
888,0.528167,0.53258,0.185556,0.308755,7.901981


In [52]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1031, … 1756]",4
1,"[1, 1553, … 844]",5
2,"[2, 111, 914]",3
3,"[3, 1365, 444]",3
4,"[4, 1627, … 1572]",5


# Using PDS Expressions On Series / NumPy arrays

In [53]:
df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
)
df.head()

actual,predicted,0-2,0-9
f64,f64,i32,i32
0.0,0.651738,2,3
1.0,0.006976,1,8
0.0,0.387053,1,2
1.0,0.631486,1,6
1.0,0.693925,1,7


In [54]:
pds.eval_series(
    df["0-2"], df["0-9"], # use series as args
    expr = "jaccard_col" # name of the pds expression
)

jaccard_col
f64
0.3


In [55]:
pds.eval_series(
    df["actual"], df["predicted"], # use series as args
    expr = "query_binary_metrics" # name of the pds expression
).unnest("binary_metrics")

precision,recall,f,average_precision,roc_auc
f64,f64,f64,f64,f64
0.503837,0.504773,0.504305,0.502323,0.502156


In [56]:
pds.eval_series(
    # can also use NumPy
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    expr = "psi", # name of the pds expression
    n_bins = 5, 
    return_report = True
).unnest("psi")

cnt<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.170678,0.2,0.172,0.004223
0.362995,0.2,0.196,0.000081
0.571221,0.2,0.207,0.000241
0.774435,0.2,0.203,0.000045
inf,0.2,0.222,0.002296


In [57]:
pds.eval_series(
    np.random.random(size = 1000), 
    expr = "query_cid_ce", # name of the pds expression
)

cid_ce
f64
12.312627


In [58]:
pds.eval_series(
    pl.Series(values=np.random.random(size = 1000)), 
    expr = "query_c3_stats", # name of the pds expression
    lag = 3
)

c3_stats
f64
0.13283
